- csv と transformers ライブラリがインポートされ、transformers ライブラリから BertTokenizer と BertModel クラスを使用して BERT トークナイザとモデルがロードされています。
- 2212.csv ファイルが読み取りモードで開かれ、ファイルを読み取るために CSV リーダー オブジェクトが作成されます。
- ヘッダ行は next 関数を用いてスキップされます。
- CSV ファイルの行を繰り返し処理するために、for ループが使用されます。各行について、日付、文、ユーザ名の値が抽出される。
- 文は、BERT トークナイザーを使用して BERT 入力として符号化され、文の BERT 埋め込みは、BERT モデルを使用して計算されます。
-  文の BERT 埋め込みは、モデル出力から抽出される。
- 文の埋め込みと参照埋め込みとの間のコサイン類似度は、PyTorchのnnモジュールのCosineSimilarityクラスを使って計算される。
- 日付、文、ユーザ名、類似度の値が出力される。

In [1]:
import pandas as pd
import torch
from transformers import BertJapaneseTokenizer, BertModel
import re

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/221212_ann_mon.csv"
df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)

df
# df.to_csv("sorted.csv", index=False)

,created_at,text
0,2022-12-13 01:00:00,#cnann
1,2022-12-13 01:00:00,#cnann
2,2022-12-13 01:00:00,#cnann
3,2022-12-13 01:00:00,#cnann
4,2022-12-13 01:00:00,#cnann
...,...,...
3193,2022-12-13 03:03:53,アルニキ、松ニキお疲れ様。 #cnann
3194,2022-12-13 03:03:53,来週、そこには劣等感が火を吹いて燃え尽きたDJニキが… #cnann
3195,2022-12-13 03:03:54,IOさんとKEIJUさんの呼び方はどうなるのか…。 そしてそのまま呼べるのか…。 お疲れ様で...
3196,2022-12-13 03:04:03,今日も楽しかったです😊 お疲れ様でした！ また来週〜👋 #cnann


In [3]:
def remove_urls_and_hashtags(text):
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    # Remove words starting with "#"
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'＃\S+', '', text)
    return text


df['text'] = df['text'].apply(remove_urls_and_hashtags)
df
# df.to_csv('filtered.csv', index=False)


,created_at,text
0,2022-12-13 01:00:00,
1,2022-12-13 01:00:00,
2,2022-12-13 01:00:00,
3,2022-12-13 01:00:00,
4,2022-12-13 01:00:00,
...,...,...
3193,2022-12-13 03:03:53,アルニキ、松ニキお疲れ様。
3194,2022-12-13 03:03:53,来週、そこには劣等感が火を吹いて燃え尽きたDJニキが…
3195,2022-12-13 03:03:54,IOさんとKEIJUさんの呼び方はどうなるのか…。 そしてそのまま呼べるのか…。 お疲れ様で...
3196,2022-12-13 03:04:03,今日も楽しかったです😊 お疲れ様でした！ また来週〜👋


In [4]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_texts_by_minute = pd.DataFrame({
    "texts_by_minute": groups.apply(lambda x: x["text"].tolist())
})
df_texts_by_minute = df_texts_by_minute.reset_index()
df_texts_by_minute
# df_texts_by_minute.to_csv("texts_by_minute.csv", index=False)

,created_at,texts_by_minute
0,2022-12-13 01:00:00,"[, , , , , , , , , , , , , , , , , , , , Creep..."
1,2022-12-13 01:01:00,"[, 今年の漢字は 『 戰 』 , , , , 今年の漢字「戦」 , , , , , , ..."
2,2022-12-13 01:02:00,"[, , , , , 韻は無敵すぎw , 今年の漢字「戦」 , 今年の漢字…「韻」でしの..."
3,2022-12-13 01:03:00,"[今年の漢字 やっぱりきた！ , 今年の漢字、2人はずっと皿と韻 , 繋 はオフィシャル過..."
4,2022-12-13 01:04:00,"[真摯に向き合った白紙 , 韻、皿、擦、繋。 , 今年の漢字 Rさん「韻」 松永さん「皿」..."
...,...,...
120,2022-12-13 03:00:00,"[お疲れ様でした😀 来週のSPW楽しみにしてます!! , お疲れっしたー わちゃわちゃ良かっ..."
121,2022-12-13 03:01:00,"[2人だけ回お疲れ様でした！ , お疲れ様でした～ , お疲れ様でした！おやすみなさい！ ,..."
122,2022-12-13 03:02:00,"[お疲れ様でした〜✨ , クリピニキ, R-指定と、俺！ お疲れさまでしたー❤ , おれ..."
123,2022-12-13 03:03:00,"[超楽しかった！ お疲れさまでした , 来週のSWは色んな意味で楽しみです。 , アルニキ、..."


In [5]:
def sentence_to_vector(model, tokenizer, sentence):
    tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
    input = torch.tensor(tokens).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input, output_hidden_states=True)
        last_hidden_state = outputs[0][:, 0, :]
        averaged_hidden_state = last_hidden_state.mean(dim=0).unsqueeze(0)
    return averaged_hidden_state


def calc_similarity(sentence1, sentence2):
    print("{}\n{}".format(sentence1, sentence2))

    sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
    sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

    # Reshape the tensors to 1D
    sentence_vector1 = sentence_vector1.reshape(-1)
    sentence_vector2 = sentence_vector2.reshape(-1)

    similarity = float(torch.nn.functional.cosine_similarity(
        sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy())
    print("Similarity:", similarity)

    return similarity


def calc_average_similarity(sentences):
    similarities = []
    for i in range(1, len(sentences)):
        similarity = calc_similarity(sentences[0], sentences[i])
        similarities.append(similarity)
    average_similarity = sum(similarities) / len(similarities)
    # print("Similarities:", similarities)
    print("Average similarity:", average_similarity)

    return average_similarity

In [6]:
df_texts_by_minute["average_similarity"] = df_texts_by_minute["texts_by_minute"].apply(
    calc_average_similarity)
df_texts_by_minute




Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209

Creepy Nutsのオールナイトニッポン | ニッポン放送 | 2022/12/12/月  25:00-27:00     
Similarity: 0.7319310307502747


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity

,created_at,texts_by_minute,average_similarity
0,2022-12-13 01:00:00,"[, , , , , , , , , , , , , , , , , , , , Creep...",0.987963
1,2022-12-13 01:01:00,"[, 今年の漢字は 『 戰 』 , , , , 今年の漢字「戦」 , , , , , , ...",0.917735
2,2022-12-13 01:02:00,"[, , , , , 韻は無敵すぎw , 今年の漢字「戦」 , 今年の漢字…「韻」でしの...",0.776079
3,2022-12-13 01:03:00,"[今年の漢字 やっぱりきた！ , 今年の漢字、2人はずっと皿と韻 , 繋 はオフィシャル過...",0.737548
4,2022-12-13 01:04:00,"[真摯に向き合った白紙 , 韻、皿、擦、繋。 , 今年の漢字 Rさん「韻」 松永さん「皿」...",0.687221
...,...,...,...
120,2022-12-13 03:00:00,"[お疲れ様でした😀 来週のSPW楽しみにしてます!! , お疲れっしたー わちゃわちゃ良かっ...",0.763214
121,2022-12-13 03:01:00,"[2人だけ回お疲れ様でした！ , お疲れ様でした～ , お疲れ様でした！おやすみなさい！ ,...",0.829241
122,2022-12-13 03:02:00,"[お疲れ様でした〜✨ , クリピニキ, R-指定と、俺！ お疲れさまでしたー❤ , おれ...",0.770184
123,2022-12-13 03:03:00,"[超楽しかった！ お疲れさまでした , 来週のSWは色んな意味で楽しみです。 , アルニキ、...",0.832946


In [8]:
df_texts_by_minute.to_csv('cosine_similarity.csv', index=False)